In [2]:
# Importação dos dados:
import pandas as pd

base = pd.read_csv('dataset.csv')
base

,Status de Pagamento,Dia,Mês,Meio de Propaganda,Valor Pago,Chamadas Realizadas,Duração da chamada,Modelo de Treinamento,Nivel de Treinamento,Código de Área,Equipe,Consultor
0,Pago,1,Abr,Televisão,R$ 7000000,3,02:00,GK,KJI. L4,A7,Equipe 1,Arthur
1,Pago,10,Ago,Televisão,R$ 11000000,1,02:00,GK,Pre. L8,B13,Equipe 2,Heitor
2,Pago,20,Set,Website,R$ 12000000,3,02:00,GK,Fndn. L5,A1,Equipe 2,Helena
3,Pago,23,Set,Facebook,R$ 15000000,1,02:00,GK,Fndn. L5,A2,Equipe 3,Alice
4,Pago,11,Set,Televisão,R$ 25000000,2,02:00,GK,Fndn. L3,B13,Equipe 4,Theo
...,...,...,...,...,...,...,...,...,...,...,...,...
1232,Não pago,18,Nov,Facebook,R$ 0,2,12:55,NaN,NaN,A1,Equipe 4,Theo
1233,Não pago,24,Nov,WhatsApp,R$ 0,2,12:55,NaN,NaN,A7,Equipe 2,Heitor
1234,Não pago,9,Dez,Televisão,R$ 0,1,12:55,NaN,NaN,A4,Equipe 2,Laura
1235,Não pago,11,Fev,Facebook,R$ 0,2,12:55,NaN,NaN,B18,Equipe 2,Laura


In [24]:
# Tratamento dos valores:

base.loc[base['Mês'] == 'Jan', 'Mês'] = 1
base.loc[base['Mês'] == 'Fev', 'Mês'] = 2
base.loc[base['Mês'] == 'Mar', 'Mês'] = 3
base.loc[base['Mês'] == 'Abr', 'Mês'] = 4
base.loc[base['Mês'] == 'Mai', 'Mês'] = 5
base.loc[base['Mês'] == 'Jun', 'Mês'] = 6
base.loc[base['Mês'] == 'Jul', 'Mês'] = 7
base.loc[base['Mês'] == 'Ago', 'Mês'] = 8
base.loc[base['Mês'] == 'Set', 'Mês'] = 9
base.loc[base['Mês'] == 'Out', 'Mês'] = 10
base.loc[base['Mês'] == 'Nov', 'Mês'] = 11
base.loc[base['Mês'] == 'Dez', 'Mês'] = 12


base['Chamadas Realizadas']=base['Chamadas Realizadas'].astype(int)

base.loc[base['Status de Pagamento'] == 'Pago', 'Status de Pagamento'] = 1
base.loc[base['Status de Pagamento'] == 'Não pago', 'Status de Pagamento'] = 0 
base['Status de Pagamento'] = base['Status de Pagamento'].astype(int)

base['Valor Pago'] = base['Valor Pago'].str.lstrip('R$ ')
base['Valor Pago'] = base['Valor Pago'].astype(int)
